**This Script is for the WEFE Indicators calculation, assigning the values to each grid cell.**

The main worlflow will be:


1. Loads your grid polygon, and indicator tables.

2. Uses a metadata dictionary to determine how each indicator should be assigned (population-weighted, area-weighted, basin-level, climatic-zone-level, or national constant).

3. Supports multiple pillars (Water, Energy, Food, Ecosystem) dynamically.

4. Handles raster-based weighting, normalizatio and simple aggregation.

5. Outputs the enriched grid ready for Power BI or further analysis.


In [2]:
import geopandas as gpd
import pandas as pd
!pip install rasterio
import rasterio
import rasterio.mask
import numpy as np
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 87.8 MB/s eta 0:00:00


In [3]:
from pathlib import Path
pd.set_option("display.max_columns", 200)

1. Load the grid cells of Tunisia and set the user inputs

In [4]:
grid = gpd.read_file('/content/drive/MyDrive/TUN_wefe_PowerBI/TUN_cells_WGS84.shp')
outputs = "/content/drive/MyDrive/TUN_wefe_PowerBI/Outputs"
year = 2019
pop_col = f"pop_{year}"

import os
os.makedirs(outputs, exist_ok = True)
print("Will write outputs to:", outputs)

Will write outputs to: /content/drive/MyDrive/TUN_wefe_PowerBI/Outputs


2. Load water basin and climatic zone shapefiles

In [5]:
basins = gpd.read_file('/content/drive/MyDrive/TUN_wefe_PowerBI/TUN_WaterBasin_level04.shp')
climatic_zones = gpd.read_file('/content/drive/MyDrive/TUN_wefe_PowerBI/TUN_ClimaticZone.shp')

3. Load the tablur data

In [6]:
water_csv = "/content/drive/MyDrive/TUN_wefe_PowerBI/water_indicators.csv"
energy_csv = "/content/drive/MyDrive/TUN_wefe_PowerBI/energy_indicators.csv"
food_csv = "/content/drive/MyDrive/TUN_wefe_PowerBI/food_indicators.csv"

Load indicator tables if they exist

In [39]:
def try_load_csv(path):
    p = Path(path)
    if p.exists():
        # Add delimiter=';' to handle semicolon-separated files
        df = pd.read_csv(p, delimiter=';')
        print(f"Loaded {p.name} with shape {df.shape}")
        return df
    else:
        print(f"Skipping missing file: {p}")
        return None

In [40]:
water_df = try_load_csv(water_csv)
energy_df = try_load_csv(energy_csv)
food_df = try_load_csv(food_csv)

Loaded water_indicators.csv with shape (5, 9)
Loaded energy_indicators.csv with shape (5, 8)
Loaded food_indicators.csv with shape (5, 10)


Attach a pointer so we can reference tables by name from metadata

In [42]:
tables = {
    'water': water_df,
    'energy':energy_df,
    'food': food_df
}

4. Define metadata for assignment

In [52]:
from pathlib import Path

starter_meta = pd.DataFrame([
     # --- WATER ---
    # population-weighted national percentages (same % per cell; use pop for aggregations)
    dict(indicator='pct_basic_drinking_water', pillar='Water', assignment='population', direction='pos',
         table='water', value_col='pct_basic_drinking_water', zone_key='', basin_key='',
         notes='National %; per-cell percentage is constant; use population for weighted aggregation'),
    dict(indicator='pct_basic_sanitation', pillar='Water', assignment='population', direction='pos',
         table='water', value_col='pct_basic_sanitation', zone_key='', basin_key='',
         notes='National %; population-weighted for aggregations'),
    # basin-level (if water table has basin values; otherwise treated as national constant)
    dict(indicator='iwrm_implementation_score', pillar='Water', assignment='basin', direction='pos',
         table='water', value_col='iwrm_implementation_score', zone_key='', basin_key='HYBAS_ID',
         notes='If table lacks basin_id column, will fall back to national constant'),
    dict(indicator='annual_freshwater_withdrawals_pct', pillar='Water', assignment='national', direction='neg',
         table='water', value_col='annual_freshwater_withdrawals_pct', zone_key='', basin_key='',
         notes='National constant; higher is generally worse (pressure) → direction=neg'),
    dict(indicator='renewable_internal_freshwater_per_capita', pillar='Water', assignment='population', direction='pos',
         table='water', value_col='renewable_internal_freshwater_per_capita', zone_key='', basin_key='',
         notes='Per-capita national; use population for aggregations'),
    dict(indicator='environmental_flow_requirements_million_m3', pillar='Water', assignment='basin', direction='pos',
         table='water', value_col='environmental_flow_requirements_million_m3', zone_key='', basin_key='HYBAS_ID',
         notes='Assign per basin_id; same value to all basin cells'),
    dict(indicator='avg_precipitation_mm', pillar='Water', assignment='area', direction='pos',
         table='water', value_col='avg_precipitation_mm', zone_key='', basin_key='',
         notes='If provided as a single national/zonal figure, assigns constant; else supply zonal table'),

     # --- ENERGY --- (examples; edit as you have data)
    dict(indicator='access_to_electricity_pct', pillar='Energy', assignment='population', direction='pos',
         table='energy', value_col='access_to_electricity_pct', zone_key='', basin_key='',
         notes='Population-weighted national/subnational percentage'),
    dict(indicator='electric_power_consumption_kwh_per_capita', pillar='Energy', assignment='population', direction='pos',
         table='energy', value_col='electric_power_consumption_kwh_per_capita', zone_key='', basin_key='',
         notes='Per-capita; use population for aggregations'),
    dict(indicator='renewable_energy_consumption_pct', pillar='Energy', assignment='climatic_zone', direction='pos',
         table='energy', value_col='renewable_energy_consumption_pct', zone_key='clima_zone', basin_key='',
         notes='Assign per climatic zone or national'),
    dict(indicator='renewable_electricity_output_pct', pillar='Energy', assignment='national', direction='pos',
         table='energy', value_col='renewable_electricity_output_pct', zone_key='', basin_key='',
         notes='National constant'),
    dict(indicator='energy_imports_net_pct', pillar='Energy', assignment='national', direction='neg',
         table='energy', value_col='energy_imports_net_pct', zone_key='', basin_key='',
         notes='Macro-level pressure → direction=neg'),
    dict(indicator='co2_emissions_per_capita', pillar='Energy', assignment='population', direction='neg',
         table='energy', value_col='co2_emissions_per_capita', zone_key='', basin_key='',
         notes='Population-weighted; higher = worse → direction=neg'),

      # --- FOOD --- (examples; edit as you have data)
    dict(indicator='undernourishment_pct', pillar='Food', assignment='population', direction='neg',
         table='food', value_col='undernourishment_pct', zone_key='', basin_key='',
         notes='Higher is worse → direction=neg'),
    dict(indicator='avg_protein_supply_grams_per_capita_per_day', pillar='Food', assignment='population', direction='pos',
         table='food', value_col='avg_protein_supply_grams_per_capita_per_day', zone_key='', basin_key='',
         notes='Per-capita national'),
    dict(indicator='child_wasting_pct', pillar='Food', assignment='population', direction='neg',
         table='food', value_col='child_wasting_pct', zone_key='', basin_key='',
         notes='If subnational, add a zone_key column and values per zone'),
    dict(indicator='child_stunting_pct', pillar='Food', assignment='population', direction='neg',
         table='food', value_col='child_stunting_pct', zone_key='', basin_key='',
         notes='Same'),
    dict(indicator='adult_obesity_pct', pillar='Food', assignment='population', direction='neg',
         table='food', value_col='adult_obesity_pct', zone_key='', basin_key='',
         notes='Higher obesity = worse'),
    dict(indicator='adesa_pct', pillar='Food', assignment='national', direction='pos',
         table='food', value_col='adesa_pct', zone_key='', basin_key='',
         notes='National constant'),
    dict(indicator='avg_food_production_value_per_capita', pillar='Food', assignment='population', direction='pos',
         table='food', value_col='avg_food_production_value_per_capita', zone_key='', basin_key='',
         notes='Per-capita; population-weighted'),
    dict(indicator='cereal_yield_kg_per_hectare', pillar='Food', assignment='crop_area', direction='pos',
         table='food', value_col='cereal_yield_kg_per_hectare', zone_key='', basin_key='',
         notes='Area-weighted on cropland only via pct_crop'),

])

INDICATOR_META_CSV = "/content/drive/MyDrive/TUN_wefe_PowerBI/indicator_metadata.csv"
meta_path = Path(INDICATOR_META_CSV)
meta_path.parent.mkdir(parents=True, exist_ok=True)
if not meta_path.exists():
    starter_meta.to_csv(meta_path, index=False)
    print(f"Starter metadata written to: {meta_path}")
else:
    print(f"Using existing metadata at: {meta_path}")

meta = pd.read_csv(meta_path)
display(meta.head(12))


Starter metadata written to: /content/drive/MyDrive/TUN_wefe_PowerBI/indicator_metadata.csv


,indicator,pillar,assignment,direction,table,value_col,zone_key,basin_key,notes
0,pct_basic_drinking_water,Water,population,pos,water,pct_basic_drinking_water,NaN,NaN,National %; per-cell percentage is constant; u...
1,pct_basic_sanitation,Water,population,pos,water,pct_basic_sanitation,NaN,NaN,National %; population-weighted for aggregations
2,iwrm_implementation_score,Water,basin,pos,water,iwrm_implementation_score,NaN,HYBAS_ID,"If table lacks basin_id column, will fall back..."
3,annual_freshwater_withdrawals_pct,Water,national,neg,water,annual_freshwater_withdrawals_pct,NaN,NaN,National constant; higher is generally worse (...
4,renewable_internal_freshwater_per_capita,Water,population,pos,water,renewable_internal_freshwater_per_capita,NaN,NaN,Per-capita national; use population for aggreg...
5,environmental_flow_requirements_million_m3,Water,basin,pos,water,environmental_flow_requirements_million_m3,NaN,HYBAS_ID,Assign per basin_id; same value to all basin c...
6,avg_precipitation_mm,Water,area,pos,water,avg_precipitation_mm,NaN,NaN,"If provided as a single national/zonal figure,..."
7,access_to_electricity_pct,Energy,population,pos,energy,access_to_electricity_pct,NaN,NaN,Population-weighted national/subnational perce...
8,electric_power_consumption_kwh_per_capita,Energy,population,pos,energy,electric_power_consumption_kwh_per_capita,NaN,NaN,Per-capita; use population for aggregations
9,renewable_energy_consumption_pct,Energy,climatic_zone,pos,energy,renewable_energy_consumption_pct,clima_zone,NaN,Assign per climatic zone or national


5. Helper functions(Assignments, Normalization, Aggregation)

In [50]:

def assign_indicator_to_grid(grid_df, tables, meta_row, year, pop_col):
    """Assign one indicator to the grid based on metadata rules."""
    indicator = meta_row['indicator']
    value_col = meta_row['value_col']
    table_key  = str(meta_row['table']).lower() if pd.notna(meta_row['table']) else ''
    assignment = meta_row['assignment']
    zone_key   = meta_row['zone_key'] if pd.notna(meta_row['zone_key']) else ''
    basin_key  = meta_row['basin_key'] if pd.notna(meta_row['basin_key']) else ''

    df = tables.get(table_key)
    if df is None:
        # If no table, create a NaN column
        grid_df[indicator] = np.nan
        return grid_df

    row = pick_year_row(df, year)
    # For national constants or single-row tables, row may have only value columns

    g = grid_df.copy()

    if assignment == 'population':
        # Assign the same % or value to each cell; keep a helper 'weighted' column for aggregations
        val = row[value_col].iloc[0]
        g[indicator] = val
        # Optional: a people-based count for aggregations
        if pop_col in g.columns:
            g[indicator + "_people_equiv"] = g[pop_col] * (val / 100.0 if val > 1 else val)
        return g

    elif assignment == 'national':
        # Single national constant to all cells
        val = row[value_col].iloc[0]
        g[indicator] = val
        return g

    elif assignment == 'basin':
        # Expect a table with basin values (column matching grid 'HYBAS_ID' or meta basin_key)
        key = basin_key if basin_key else 'HYBAS_ID'
        if key not in g.columns:
            # Updated to use the correct key in the error message
            raise ValueError(f"Grid missing '{key}' for basin assignment.")
        # If the table doesn't have basin ids, treat as national constant
        possible_keys = [key, 'HYBAS_ID', 'basin_id', 'basin', 'basin_code']
        match_key = next((k for k in possible_keys if k in row.columns), None)
        if match_key is None:
            # No basin column present in the table
            val = row[value_col].iloc[0]
            g[indicator] = val
            return g
        tmp = row[[match_key, value_col]].drop_duplicates()
        g = g.merge(tmp, left_on=key, right_on=match_key, how='left')
        g[indicator] = g[value_col]
        g = g.drop(columns=[value_col, match_key])
        return g

    elif assignment == 'climatic_zone':
        key = zone_key if zone_key else 'clima_zone'
        if key not in g.columns:
            # Updated to use the correct key in the error message
            raise ValueError(f"Grid missing '{key}' for zone assignment.")
        possible_keys = [key, 'clima_zone', 'zone_id', 'climatic_zone_code', 'zone']
        match_key = next((k for k in possible_keys if k in row.columns), None)
        if match_key is None:
            # No zone column present in the table → treat as national constant
            val = row[value_col].iloc[0]
            g[indicator] = val
            return g
        tmp = row[[match_key, value_col]].drop_duplicates()
        g = g.merge(tmp, left_on=key, right_on=match_key, how='left')
        g[indicator] = g[value_col]
        g = g.drop(columns=[value_col, match_key])
        return g

    elif assignment in ('area','crop_area','forest_area','sum'):
        # These assume the value is already provided at grid or is a single national value.
        # If it's a single national/scalar, copy it to all; otherwise, if the column exists on the grid, use it.
        val = row[value_col].iloc[0]
        g[indicator] = val
        # If using crop/forest area weighting in aggregation, we will use pct_crop / pct_forest as weights later
        return g

    else:
        # Fallback: copy constant
        val = row[value_col].iloc[0]
        g[indicator] = val
        return g

def minmax_normalize(series, direction='pos'):
    s = series.astype(float)
    if s.notna().sum() <= 1:
        # Not enough variation; return 0.5 for non-null, else NaN
        return s.where(s.isna(), 0.5)
    s_min, s_max = s.min(), s.max()
    if s_max == s_min:
        norm = pd.Series(0.5, index=s.index)
    else:
        norm = (s - s_min) / (s_max - s_min)
    if str(direction).lower() == 'neg':
        norm = 1 - norm
    return norm

def weighted_mean(series, weights):
    w = weights.fillna(0).astype(float)
    s = series.astype(float)
    denom = w.sum()
    if denom == 0:
        return np.nan
    return (s * w).sum() / denom

Apply Assignments for all Metadata Indicators

In [45]:
def pick_year_row(df, year):
    if df is None:
        return None
    if 'data_year' not in df.columns:
        # assume a single-row table of constants
        return df.iloc[[0]]
    sel = df[df['data_year'] == year]
    if sel.empty:
        # fallback to latest available
        sel = df.sort_values('data_year').iloc[[-1]]
        print(f" Using latest available year {int(sel['data_year'].iloc[0])} for a table.")
    return sel

def safe_merge(left, right, left_key, right_key, how='left'):
    if right is None or right.empty:
        return left.copy()
    return left.merge(right[[right_key] + [c for c in right.columns if c not in ['data_year']]],
                      left_on=left_key, right_on=right_key, how=how)

In [46]:
enriched = grid.copy()

# Ensure float % cover columns are in [0,1] for weighting convenience later
for c in ['pct_crop','pct_forest']:
    if c in enriched.columns and enriched[c].max() > 1.0:
        # assume 0-100%
        enriched[c] = enriched[c] / 100.0

In [53]:
# Apply assignments
for _, row in meta.iterrows():
    ind = row['indicator']
    table_key = str(row['table']).lower()
    if table_key not in tables:
        print(f"Skipping {ind}: table '{table_key}' not among {list(tables.keys())}")
        continue
    enriched = assign_indicator_to_grid(enriched, tables, row, year, pop_col)

print("Assigned indicators added to grid. Current columns:")
print([c for c in enriched.columns if c not in grid.columns])

Assigned indicators added to grid. Current columns:
['pct_basic_drinking_water', 'pct_basic_drinking_water_people_equiv', 'pct_basic_sanitation', 'pct_basic_sanitation_people_equiv', 'annual_freshwater_withdrawals_pct', 'renewable_internal_freshwater_per_capita', 'renewable_internal_freshwater_per_capita_people_equiv', 'avg_precipitation_mm', 'access_to_electricity_pct', 'access_to_electricity_pct_people_equiv', 'electric_power_consumption_kwh_per_capita', 'electric_power_consumption_kwh_per_capita_people_equiv', 'iwrm_implementation_score', 'environmental_flow_requirements_million_m3', 'renewable_energy_consumption_pct', 'renewable_electricity_output_pct', 'energy_imports_net_pct', 'co2_emissions_per_capita', 'co2_emissions_per_capita_people_equiv', 'undernourishment_pct', 'undernourishment_pct_people_equiv', 'avg_protein_supply_grams_per_capita_per_day', 'avg_protein_supply_grams_per_capita_per_day_people_equiv', 'child_wasting_pct', 'child_wasting_pct_people_equiv', 'child_stunting_

Normalize, Build Pillar Score, and WEFE Nexus

In [56]:
# Group indicators by pillar from metadata
pillar_to_inds = (meta.groupby('pillar')['indicator']
                    .apply(list)
                    .to_dict())

# Normalize each indicator per its direction
for _, row in meta.iterrows():
    ind = row['indicator']
    direction = row['direction']
    col = ind
    if col in enriched.columns:
        enriched[ind + "_norm"] = minmax_normalize(enriched[col], direction=direction)
    else:
        print(f"Missing indicator column for normalization: {ind}")

# Compute pillar scores as simple mean of normalized indicators present
for pillar, inds in pillar_to_inds.items():
    norm_cols = [f"{i}_norm" for i in inds if f"{i}_norm" in enriched.columns]
    if not norm_cols:
        enriched[pillar + "_score"] = np.nan
    else:
        enriched[pillar + "_score"] = enriched[norm_cols].mean(axis=1)

# Correctly identify only the composite pillar scores
pillar_scores = [f"{p}_score" for p in pillar_to_inds.keys()]
# Filter out any pillar scores that didn't get a column
pillar_scores = [p for p in pillar_scores if p in enriched.columns]
enriched['WEFE_Nexus'] = enriched[pillar_scores].mean(axis=1)

print("Pillar columns:", pillar_scores)

Pillar columns: ['Energy_score', 'Food_score', 'Water_score']


Aggregations for PowerBI (National, Climatic Zones, Basins)

In [59]:
# Choose weighting scheme for aggregations
# For population-weighted aggregation, use POP_COL
# For area-weighted, use 'area_m2'
# For crop/forest-specific, you can customize per indicator.

main_pillars = ['Water', 'Energy', 'Food']
pillar_score_cols = [f'{p}_score' for p in main_pillars]

def aggregate_scores(df, by=None, weight_col=None):
    # This list will now only contain the true pillar scores
    scores_to_aggregate = [c for c in pillar_score_cols if c in df.columns]

    if by is None:
        # national
        if weight_col and weight_col in df.columns:
            wt = df[weight_col]
            row = { 'level': 'national_weighted',
                    'WEFE_Nexus': weighted_mean(df['WEFE_Nexus'], wt) }
            for p in scores_to_aggregate:
                row[p] = weighted_mean(df[p], wt)
            return pd.DataFrame([row])
        else:
            row = { 'level': 'national_mean',
                    'WEFE_Nexus': df['WEFE_Nexus'].mean() }
            for p in scores_to_aggregate:
                row[p] = df[p].mean()
            return pd.DataFrame([row])
    else:
        grp = df.groupby(by)
        out = []
        for key, sub in grp:
            if weight_col and weight_col in sub.columns:
                wt = sub[weight_col]
                rec = { by: key, 'WEFE_Nexus': weighted_mean(sub['WEFE_Nexus'], wt) }
                for p in scores_to_aggregate:
                    rec[p] = weighted_mean(sub[p], wt)
            else:
                rec = { by: key, 'WEFE_Nexus': sub['WEFE_Nexus'].mean() }
                for p in scores_to_aggregate:
                    rec[p] = sub[p].mean()
            out.append(rec)
        return pd.DataFrame(out)

# National (population-weighted)
agg_national = aggregate_scores(enriched, by=None, weight_col=pop_col)

# By climatic zone (population-weighted)
if 'climatic_zone_code' in enriched.columns:
    agg_zone = aggregate_scores(enriched, by='climatic_zone_code', weight_col=pop_col)
else:
    agg_zone = pd.DataFrame()

# By basin (population-weighted)
if 'basin_id' in enriched.columns:
    agg_basin = aggregate_scores(enriched, by='basin_id', weight_col=pop_col)
else:
    agg_basin = pd.DataFrame()

agg_national, agg_zone.head(), agg_basin.head()


(               level  WEFE_Nexus  Water_score  Energy_score  Food_score
 0  national_weighted         0.5          0.5           0.5         0.5,
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [])

Export for PowerBI

NameError: name 'agg_basin' is not defined

In [62]:
# Export enriched grid attributes as CSV (no geometry) for Power BI
attr_cols = ['id','HYBAS_ID','clima_zone', pop_col, 'area_m2','pct_forest','pct_crop'] +             [c for c in enriched.columns if c.endswith('_norm') or c.endswith('_score') or c == 'WEFE_Nexus'] +             [c for c in enriched.columns if c in meta['indicator'].tolist()]

attr_cols = [c for c in attr_cols if c in enriched.columns]  # keep only existing

grid_attr = enriched[attr_cols].copy()
grid_attr.to_csv(f"{outputs}/wefe_grid_attributes_{year}.csv", index=False)

# Export rollups
agg_national.to_csv(f"{outputs}/wefe_national_{year}.csv", index=False)
if not agg_zone.empty:
    agg_zone.to_csv(f"{outputsR}/wefe_climatic_zone_{year}.csv", index=False)
if not agg_basin.empty:
    agg_basin.to_csv(f"{outputs}/wefe_basin_{year}.csv", index=False)

# Optional: Export geometry (GeoJSON) with id + scores (convert to TopoJSON via mapshaper.org)
geo_cols = ['id','WEFE_Nexus'] + [c for c in enriched.columns if c.endswith('_score')]
geo_cols = [c for c in geo_cols if c in enriched.columns]
enriched_geo = enriched[geo_cols + ['geometry']].copy()
out_geo = f"{outputs}/wefe_grid_{year}.geojson"
enriched_geo.to_file(out_geo, driver='GeoJSON')

print("✔ Exports written:")
print("-", f"{outputs}/wefe_grid_attributes_{year}.csv")
print("-", f"{outputs}/wefe_national_{year}.csv")
if not agg_zone.empty:
    print("-", f"{outputs}/wefe_climatic_zone_{year}.csv")
if not agg_basin.empty:
    print("-", f"{outputs}/wefe_basin_{year}.csv")
print("-", out_geo, "(convert to TopoJSON for Power BI Shape Map)")


✔ Exports written:
- /content/drive/MyDrive/TUN_wefe_PowerBI/Outputs/wefe_grid_attributes_2019.csv
- /content/drive/MyDrive/TUN_wefe_PowerBI/Outputs/wefe_national_2019.csv
- /content/drive/MyDrive/TUN_wefe_PowerBI/Outputs/wefe_grid_2019.geojson (convert to TopoJSON for Power BI Shape Map)
